In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Display 1st 5 rows of Dataframe so I can see how the data was organized in the .csv
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
### Player Count
# Finding the number of unique players
player_count = len(purchase_data["SN"].unique())
# Creating Player Count summary dataframe
player_count_d = {'Total Players': player_count}
player_count_df = pd.DataFrame(data = player_count_d, index = [0])
player_count_df

,Total Players
0,576


In [4]:
### Purchasing Analysis (Total)
# Finding the Number of Unique Items
unique_items = len(purchase_data["Item Name"].unique())

# Finding the Average Purchase Price
avg_price = purchase_data["Price"].mean()

# Finding the Total Number of Purchases
total_purchases = len(purchase_data["Price"])

# Finding the Total Revenue
total_revenue = purchase_data["Price"].sum()

# Outputting the Purchasing Analysis into a Summary Dataframe
# First, inputted calculated into a dictionary
purchasing_analysis_d = {'Number of Unique Items': [unique_items], \
                         'Average Purchase Price': [avg_price], \
                         'Total Number of Purchases': [total_purchases], \
                         'Total Revenue': [total_revenue]}
# Then, converted dictionary to a dataframe and added currency formatting
# to the Average Purchase Price and Total Revenue columns
purchasing_analysis_df = pd.DataFrame(data = purchasing_analysis_d)
purchasing_analysis_df.style.format({'Average Purchase Price': "${:,.2f}", \
                                   'Total Revenue': "${:,.2f}"})

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
### Gender Demographics


purchase_data_dropped = purchase_data.drop_duplicates(['SN'])
# Removed duplicate SN entries from dataframe so that the same person isn't counted multiple times below
gender = purchase_data_dropped["Gender"].value_counts().sort_index()
# Sorted in alphabetical order so that the entries and respective counts can be more easily determined

# Finding the percentage and count of male players
male_count = gender[1]
male_percentage = male_count/player_count

# Finding the percentage and count of female players
female_count = gender[0]
female_percentage = female_count/player_count

# Finding the percentage and count of other/non-disclosed
other_count = gender[2]
other_percentage = other_count/player_count

# Creating Gender Demographics summary dataframe
gender_demo_d = [["Female", female_count, female_percentage], \
                 ["Male", male_count, male_percentage], \
                 ["Other / Non-Disclosed", other_count, other_percentage]]
gender_demo_df = pd.DataFrame(gender_demo_d, columns = ['Gender', 'Total Count', 'Percentage of Players'])
gender_demo_df.set_index("Gender").style.format({"Percentage of Players": "{:,.2%}"})

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [6]:
### Purchasing Analysis (Gender)

# Finding the Purchase Count by Gender
purch_count_gender = purchase_data["Gender"].value_counts().sort_index()

# Determining the Average Purchase Price by Gender
purch_avg_gender = purchase_data.groupby("Gender")['Price'].mean()

# Finding the Total Purchase Value by Gender
purch_total_gender = purchase_data.groupby("Gender")['Price'].sum()

# Determining the Average Purchase Total per Person by Gender
purch_person_gender = [None]*len(purch_total_gender)
for i in range(0, len(purch_total_gender)):
    purch_person_gender[i] = purch_total_gender[i]/gender_demo_df.at[i, "Total Count"]
    
# Creating Purchasing Analysis (Gender) summary dataframe
genders = ['Female', 'Male', 'Other / Non Disclosed']
columns = ["Gender", 'Purchase Count', 'Average Purchase Price', 'Total Purchase Value', "Avg. Total Purchase per Person"]
gender_purchasing_df = pd.DataFrame(columns = columns)
for i in range(0, len(purch_person_gender)):
    gender_purchasing_df = gender_purchasing_df.append({"Gender": genders[i], 
                                 "Purchase Count": purch_count_gender[i], \
                                 "Average Purchase Price": purch_avg_gender[i], \
                                 "Total Purchase Value": purch_total_gender[i], \
                                 "Avg. Total Purchase per Person": purch_person_gender[i]}, ignore_index=True)
gender_purchasing_df.set_index("Gender").style.format({'Average Purchase Price': '${:,.2f}', \
                                                      'Total Purchase Value': '${:,.2f}', \
                                                      'Avg. Total Purchase per Person': '${:,.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg. Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non Disclosed,15,$3.35,$50.19,$4.56


In [7]:
### Age Demographics
# In the README.md, the instructions for this section are the same as the ones for the next Purchasing Analysis (Age) section.
# However, in HeroesOfPymoli.ipynb, the instructions and example output dataframe for this section show the count and percentages for each age
# group being calculated and displayed.
group_labels = ['<10', '10-14', '15-19', '20-24','25-29','30-34','35-39','40+']
bins = [0,10,15,20,25,30,35,40,float('inf')]

purchase_data['Age Group'] = pd.cut(purchase_data['Age'], bins, right = False, labels = group_labels)
purchase_data_dropped = purchase_data.drop_duplicates(['SN'])
# Used drop_duplicates to get rid of duplicate SNs and so that each SN is only represented once in below calculations

# Determing the Total Count and Percentage of Players for each Age Group
age_count = purchase_data_dropped['Age Group'].value_counts().sort_index()
age_percent = age_count/age_count.sum()

age_demo = pd.concat([age_count, age_percent], axis = 1)
age_demo.columns = ["Total Count", 'Percentage of Players']
age_demo.style.format({'Percentage of Players':'{:,.2%}'})

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [8]:
### Purchasing Analysis (Age)
purchase_data_group = purchase_data.groupby('Age Group')

# Determining the Purchase Count for each Age Group
purch_count_age = purchase_data_group['Price'].size()

# Determining the Average Purchase Price for each Age Group
purch_avg_age = purchase_data_group['Price'].mean()

# Determining the Total Purchase Value for each Age Group
purch_total_age = purchase_data_group['Price'].sum()

# Determining the Purchase Total per Person by Age
purch_person_age = purch_total_age/age_count

# Creating Purchasing Analysis (Age) summary dataframe
age_purchasing = pd.concat([purch_count_age, purch_avg_age, purch_total_age, purch_person_age], axis = 1)
age_purchasing.columns = ['Purchase Count', 'Average Purchase Price','Total Purchase Value', 'Avg. Total Purchase per Person']
age_purchasing.style.format({'Average Purchase Price': '${:,.2f}', \
                             'Total Purchase Value': '${:,.2f}', \
                             'Avg. Total Purchase per Person': '${:,.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg. Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [9]:
### Top Spenders
# Determining the Top 5 Spenders (the top 5 players with the greatest Total Purchase Value)
# Group the given purchase_data dataframe by SN
purchase_data_sn = purchase_data.groupby('SN')

# Find purchase counts for each screenname
purch_count = purchase_data_sn['Purchase ID'].count()
# Find the average purchase price for each screenname
purch_avg = purchase_data_sn['Price'].mean()
# Find the total purchase value for each screenname
purch_total = purchase_data_sn['Price'].sum()
# Combine sn, purch_count, purch_avg, and purch_total into a dataframe that can then be sorted by Total Purchase Value
spenders = {'Purchase Count':purch_count,'Average Purchase Price':purch_avg,'Total Purchase Value':purch_total}
spenders_df = pd.DataFrame(spenders)

# Creating Top Spenders summary Dataframe
spenders_df.sort_values(by = 'Total Purchase Value', \
                        ascending = False).head(5)\
                        .style.format({'Average Purchase Price':'${:,.2f}','Total Purchase Value':'${:,.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [10]:
### Most Popular Items
# Will determine the Top 5 Most Popular Items in a similar fashion to the Top 5 Spenders
# Top 5 Most Popular Items determined by the Purchase Count of the item
# Group the given purchase_data dataframe by Item ID
purchase_data_id = purchase_data.groupby('Item ID')

# Filter the grouped dataframe for unique item IDs
item_id = purchase_data_id['Item ID'].unique()
# Find the item names for each item ID
item_name = purchase_data_id['Item Name'].unique()
# Find the purchase counts for each item
item_count = purchase_data_id['Purchase ID'].count()
# Find the price for each item
# Some items had more than one item price, so the mean item price was calculated 
# Eg.: Crucifer was sold for $3.26 and $1.99 with mean price of $2.94
item_price = purchase_data_id['Price'].mean()
# Find the total purchase value for each item
item_tot_purch = purchase_data_id['Price'].sum()

# Creating Most Popular Items summary dataframe
items = {'Item Name':item_name, 'Purchase Count':item_count, 'Item Price':item_price, 'Total Purchase Value':item_tot_purch}
items_df = pd.DataFrame(items)

items_df.sort_values(by = 'Purchase Count', \
                    ascending = False).head(5)\
                    .style.format({'Item Price':'${:,.2f}','Total Purchase Value':'${:,.2f}'})


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,['Final Critic'],13,$4.61,$59.99
178,"['Oathbreaker, Last Hope of the Breaking Storm']",12,$4.23,$50.76
145,['Fiery Glass Crusader'],9,$4.58,$41.22
132,['Persuasion'],9,$3.22,$28.99
108,"['Extraction, Quickblade Of Trembling Hands']",9,$3.53,$31.77


In [11]:
### Most Profitable Items
# To the find the 5 most profitable items, only needed to sort the previously constructed items_df dataframe
# by the Total Purchase Value instead of Purchase Count
items_df.sort_values(by = 'Total Purchase Value', \
                    ascending = False).head(5)\
                    .style.format({'Item Price':'${:,.2f}', 'Total Purchase Value':'${:,.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,['Final Critic'],13,$4.61,$59.99
178,"['Oathbreaker, Last Hope of the Breaking Storm']",12,$4.23,$50.76
82,['Nirvana'],9,$4.90,$44.10
145,['Fiery Glass Crusader'],9,$4.58,$41.22
103,['Singed Scalpel'],8,$4.35,$34.80
